In [199]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [201]:
!pip install openpyxl

In [202]:
train_data = pd.read_csv('../input/appforg/train.csv')
test_data = pd.read_excel('../input/appforg/test.xlsx')

In [203]:
train_data.head(5)

In [204]:
result=train_data.dtypes.value_counts(normalize=True)
plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
plt.title('Checking the percentage of DataTypes')
result.plot(kind='pie',wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' },colors=['blue', 'orange', 'green'], autopct="%.0f%%",explode = (0.05, 0.05, 0.05))
plt.subplot(1,2,2)
plt.title('Checking the percentage of DataTypes')
ax = result.plot(kind='bar',figsize=(15,4),width = 0.8,color=['blue', 'orange', 'green'],edgecolor=None)
plt.xticks(fontsize=14)
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.yticks([])

for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center')
plt.show()

In [69]:
plt.figure(figsize=(10,6))
sns.displot(
    data=train_data.isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill",
    aspect=1.25
)

In [205]:
num_val = train_data.select_dtypes(exclude=['object']).copy()
cat_val = train_data.select_dtypes(include=['object']).copy()

In [206]:
num_val.head(4)

In [207]:
plt.figure(figsize=(22,24))
plt.subplot(4,2,1)
plt.xlabel('Count')
plt.ylabel('Columns')
plt.title('Counting Nan in different columns')
num_val.isnull().sum().plot(kind='bar',color='forestgreen')
plt.subplot(4,2,2)
plt.xlabel('Count')
plt.ylabel('Columns')
plt.title('Counting Nan in different columns')
cat_val.isnull().sum().plot(kind='bar',color='forestgreen')

In [208]:
nan_num_col = [col for col in num_val.columns if num_val.loc[:,col].isnull().sum()!=0]
cat_num_col = [col for col in cat_val.columns if cat_val.loc[:,col].isnull().sum()!=0]

In [212]:
for col in nan_num_col:
    train_data[col].fillna(train_data[col].median(),inplace=True) # We have too many outliers that's why using median
    test_data[col].fillna(test_data[col].median(),inplace=True)
    
for col in cat_num_col:
    train_data[col].fillna(train_data[col].value_counts().index[0], inplace=True)   # Filling the missing value with most frequent data 
    test_data[col].fillna(test_data[col].value_counts().index[0], inplace=True)
    

In [210]:
train_data.isnull().sum()

In [213]:
test_data.isnull().sum() # we cleaned it from Nans

In [74]:
plt.figure(figsize=(22,24))
for i,col in enumerate(train_data):
    if train_data.loc[:,col].dtype !='object':
        plt.subplot(6,6,i+1)
        sns.distplot(train_data.loc[:,col].dropna(),kde=True,color='blue')

*The distribution of Delay_from_due_date and Monthly_Inhand_Salary has bimodale, because it has two peaks.*
*The distribution of the variables: Total_EMI_per_month,Num_Credit_Inquiries,Interest_Rate,Num_Credit_Card and Num_Bank_Accounts is skewed, we can confirme that using skew function.*

In [214]:
num_col = train_data.select_dtypes(exclude=['object']).columns

In [215]:
skew = train_data[num_col].skew(skipna = True).sort_values(ascending=False)
skew

*As we can see all the skewness values are greater than 0, we can understand that our numerical features : Total_EMI_per_month,Num_Credit_Inquiries,Interest_Rate,Num_Credit_Card and Num_Bank_Accounts are right-skewed.*

*we can say that these variables have outliers that skew the data. we tackle it now*

In [80]:
cat_val[cat_val.duplicated()] # we don't have any duplicated values. means no row have total same values

In [216]:
plt.figure(figsize=(10,6))
plt.title('Checking the Counts of Credit Score')
train_data['Credit_Score'].value_counts().plot(kind='bar',color='red')

*So we see that Good Type of Credit Score have less counts and Standard Have the most.*

In [217]:
#print categories of each categorical column
for col in train_data.select_dtypes(exclude=['number']):
  print(f'{col:-<30},{train_data[col].unique()}')

*As we see that Age column have outliers like -500 or etc.*
*we have to clean the column Payment_Behaviour by using regex*

*Dropping Some Columns like ID,Name and some others*

In [218]:
train_data.drop(['ID','Name','Age','Customer_ID','Occupation','Changed_Credit_Limit','Credit_Mix','SSN'],axis=1,inplace=True)
test_data.drop(['ID','Name','Age','Customer_ID','Occupation','Changed_Credit_Limit','Credit_Mix','SSN'],axis=1,inplace=True)

In [219]:
new_cat_col = train_data.select_dtypes(include=['object']).columns
new_cat_col

In [221]:
year =[]
month =[]
for i,val in enumerate(train_data['Credit_History_Age']):
    x,x1,x2,x3,x4 = train_data.iloc[i]['Credit_History_Age'].split(' ',4)
    year.append(x)
    month.append(x3)
train_data['year'] = year
train_data['month'] = month

train_data.drop('Credit_History_Age',axis=1,inplace=True)


In [220]:
year =[]
month =[]
for i,val in enumerate(test_data['Credit_History_Age']):
    x,x1,x2,x3,x4 = test_data.iloc[i]['Credit_History_Age'].split(' ',4)
    year.append(x)
    month.append(x3)
test_data['year'] = year
test_data['month'] = month

test_data.drop('Credit_History_Age',axis=1,inplace=True)


In [222]:
train_data[['year','month']] = train_data[['year','month']].astype('int')

test_data[['year','month']] = test_data[['year','month']].astype('int')

In [223]:
le = LabelEncoder()
train_data['Type_of_Loan'] = le.fit_transform(train_data['Type_of_Loan'])
test_data['Type_of_Loan'] = le.fit_transform(test_data['Type_of_Loan'])

In [224]:
mon = {
    'January':0,
    'February':1,
    'March':2, 
    'April':3,
    'May':4,
    'June':5,
    'July':6,
    'August':7
}

In [225]:
train_data['Month'].replace(mon,inplace=True)
test_data['Month'].replace(mon,inplace=True)


In [226]:
train_data.drop(['Annual_Income','Payment_Behaviour'],axis=1,inplace=True)
test_data.drop(['Annual_Income','Payment_Behaviour'],axis=1,inplace=True)


In [227]:
train_data.drop(['Num_of_Loan','Num_of_Delayed_Payment','Outstanding_Debt','Payment_of_Min_Amount','Amount_invested_monthly','Monthly_Balance'],axis=1,inplace=True)
test_data.drop(['Num_of_Loan','Num_of_Delayed_Payment','Outstanding_Debt','Payment_of_Min_Amount','Amount_invested_monthly','Monthly_Balance'],axis=1,inplace=True)

In [193]:
train_data['Credit_Score'] = le.fit_transform(train_data['Credit_Score'])

In [195]:
for col in train_data.columns:
    
    if train_data[col].dtypes !='object':
        first_quartile = train_data[col].quantile(0.25)
        third_quartile = train_data[col].quantile(0.75)
        
        IQR = third_quartile - first_quartile 
        
        out = third_quartile + 3*IQR
       
        train_data.drop(train_data[train_data[col] > out].index,axis=0,inplace=True)
    

In [230]:
X = train_data.drop(['Customer_ID','Credit_Score'],axis=1)
Y = train_data.Credit_Score

In [233]:
Y = train_data.Credit_Score

In [232]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=34)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [ ]:
clf = xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.3,grow_policy='depthwise',
              importance_type=None,
              learning_rate=0.05, max_bin=25, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=15,n_estimators=100,
              verbosity=1)

In [ ]:
score = cross_val_score(clf,x_train,y_train,cv=10,error_score='raise')
score

In [ ]:
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

In [ ]:
pred = clf.predict(test_data)
sub = pd.DataFrame()
sub['Customer_ID'] = test_data['Customer_ID']
sub['Credit_Score'] = pred